In [70]:
import os
import subprocess
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver import ActionChains


from news_crawl import settings

# preferences #########################################
# scrapyの設定からpreferences情報を取得（デフォルト値）
set_preferences: dict[str,int] = settings.SELENIUM_DRIVER_SET_PREFERENCE
# 上記の値をカスタマイズしたい場合
set_preferences.update({
    # 'permissions.default.image': 2,                         # 画像のダウンロード禁止
    # 'permissions.default.image.animation_mode': 2,          # gitなどのアニメーションのダウンロード禁止
    # 'permissions.default.stylesheet': 2,                    # cssのダウンロード禁止
    # 'dom.ipc.plugins.enabled.libflashplayer.so': 'false',   # Flashを使わない
})
profile = FirefoxProfile(profile_directory='firefox_profile')   # firefoxのプロファイルが格納されているディレクトリ
# profile = FirefoxProfile()   # 広告ブロックがNG扱いされたのでプロファイル未使用、、、
for set_preference_key, set_preference_value in set_preferences.items():    # さらにプロファイルをカスタマイズする場合、set_preferenceで追加できる。
    profile.set_preference(set_preference_key, set_preference_value)


In [77]:
# options ###########################################
options = FirefoxOptions()
# options.add_argument('--headless')    # ヘッドレスモード
options.profile = profile

options.log.level = "INFO"

In [84]:
# service ###########################################
service=Service(
    executable_path= None,
    port= 0,
    service_args = ['--log', 'info'],
    # service_args = ['--log-level=INFO'], これだとエラーだった
    log_output= subprocess.DEVNULL,
    env= None,
)

In [85]:
# ドライバー：ここでブラウザが起動する ###########################################
driver = webdriver.Firefox(
    options=options,
    service=service,
)

In [86]:
driver.get("https://mainichi.jp/flash/")
# driver.get("https://jp.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml")


In [87]:
target_article_element = f'#article-list > ul > li:nth-child({20})'
WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, target_article_element)))

target_next_page_element = f'div.main-contents span.link-more'
WebDriverWait(driver, 60).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, target_next_page_element)))

<selenium.webdriver.remote.webelement.WebElement (session="aab1c854-3753-4657-bbee-f1311c4d7bdd", element="35fec2b9-f665-47a8-820d-8492e7e404e2")>

In [88]:
elem: WebElement = driver.find_element(By.CSS_SELECTOR, target_next_page_element)
print(elem.text)
print(type(elem))
action_chains = ActionChains(driver)
action_chains.pause(3)
# action_chains.reset_actions()
# action_chains.scroll_to_element(elem)
driver.execute_script('arguments[0].scrollIntoView();', elem)

# action_chains.reset_actions()
# action_chains.perform()
driver.execute_script('arguments[0].click();', elem)


# action_chains.reset_actions()
# ActionChains(driver).move_to_element(elem).pause(2).perform()
# ActionChains(driver).move_to_element(elem).pause(2).perform()
# ActionChains(driver).move_to_element(elem).pause(3)
# ActionChains(driver).pause(3).perform()
# ActionChains(driver).pause(3).perform()
# ActionChains(driver).pause(3).scroll_to_element(elem).pause(2).perform()
# ActionChains(driver).pause(3).scroll_to_element(elem)


# driver.execute_script("arguments[0].scrollIntoView();", elem)

もっと見る
<class 'selenium.webdriver.remote.webelement.WebElement'>


In [89]:
# ブラウザを閉じる
driver.quit()